In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
num_samples = 44100

In [3]:
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

In [4]:
!cat 01_simple.stride

use DesktopAudio version 1.0

signal Accum {}

loop Loop {
	ports: [
		mainInputPort InputPort {
			block: Input
		}
		mainOutputPort OutputPort {
			block: Output
		}
	]
	blocks: [
		signal Input[InputPort.size] {}
		signal Output {default: 0}
		signal Index { default: 1 reset: Reset }
		switch Done {}
		trigger Reset {}
	]
	onExecution: Reset
	terminateWhen: Done
	streams: [
		Input [ Index ] + Output >> Output ;
		Index + 1 >> Index ;
		[ Index , InputPort.size ] >> Greater () >> Done ;
		
	]
}

signal List[20] { default: 1 }

List >> Loop() >> AudioOut[1];


In [5]:
out_text = ''

accum = 0

for val1, val2 in zip(input1, input2):
    for i in range(10):
        accum += 2
    out_text += str(accum) + "\n0.0\n"

out_file = open("01_simple.expected", "w")
out_file.write(out_text[:-1])
pass

In [8]:
!cat 02_list_input.stride

use DesktopAudio version 1.0

loop Accumulate {
	ports: [	
		mainInputPort InputPort { block: Input }
		mainOutputPort OutputPort { block: Output }
	]
	blocks: [
		signal Input[InputPort.size] {}
		signal Output {default: 0 reset: Reset }
		signal Index { default: 1 reset: Reset }
		switch Done {}
		trigger Reset {}
	]
	streams: [
		Output + Input[Index] >> Output;
		Index + 1 >> Index ;
		[ Index , InputPort.size ] >> Greater () >> Done ;
	]
	onExecution: Reset
	terminateWhen: Done
}


[AudioIn[1], AudioIn[2], 7] >> Accumulate() >> AudioOut[1];



In [9]:
out_text = ''

accum = 0

for val1, val2 in zip(input1, input2):
    
    out_text += str(val1 + val2 + 7) + "\n0.0\n"

out_file = open("02_list_input.expected", "w")
out_file.write(out_text[:-1])
pass

In [10]:
!cat 03_list_input_in_module.stride

use DesktopAudio version 1.0

signal Accum {}
switch ResetOuter { }
signal OuterCounter { reset: ResetOuter}

module Sum {
	ports: [
		mainInputPort InputPort { block: Input }
		mainOutputPort OutputPort { block: Output }
	]
	blocks:  [
		signal Input[InputPort.size] { domain: Output.domain }
		loop Accumulate {
			ports: [	
				mainInputPort InputPort { block: Input }
				mainOutputPort OutputPort { block: Output }
			]
			blocks: [
				signal Input[InputPort.size] {}
				signal Output {default: 0 reset: Reset }
				signal Index { default: 1 reset: Reset }
				switch Done {}
				trigger Reset {}
			]
			streams: [
				Output + Input[Index] >> Output;
				Index + 1 >> Index ;
				[ Index , InputPort.size ] >> Greater () >> Done ;
			]
			onExecution: Reset
			terminateWhen: Done
		}
	]
	streams: [
		Input >> Accumulate() >> Output;
	]
}

[AudioIn[1], AudioIn[2], 7] >> Sum() >> AudioOut[1];



In [11]:
out_text = ''

accum = 0

for val1, val2 in zip(input1, input2):
    
    out_text += str(val1 + val2 + 7) + "\n0.0\n"

out_file = open("03_list_input_in_module.expected", "w")
out_file.write(out_text[:-1])
pass

In [12]:
!cat 04_loop_in_module.stride

use DesktopAudio version 1.0

module LoopModule {
	ports: [
		mainOutputPort OutputPort { block: Output }
	]
	blocks:  [
		signal List[20] { default: 1 }

		loop Loop {
			ports: [
				mainInputPort InputPort {
					block: Input
				}
				mainOutputPort OutputPort {
					block: Output
				}
			]
			blocks: [
				signal Input[InputPort.size] {}
				signal Output { default: 0 reset: Reset }
				signal Index { default: 1 reset: Reset }
				switch Done {}
				trigger Reset {}
			]
			onExecution: Reset
			terminateWhen: Done
			streams: [
				Input [ Index ] + Output >> Output ;
				Index + 1 >> Index ;
				[ Index , InputPort.size ] >> Greater () >> Done ;
				
			]
		}
	]
	streams: [
		List >> Loop() >> Output;
	]
}

LoopModule() >> AudioOut[1];



In [13]:
out_text = ''

counter = 0
accum = 0

for val1, val2 in zip(input1, input2):
    out_text += str(20.0) + "\n0.0\n"

out_file = open("04_loop_in_module.expected", "w")
out_file.write(out_text[:-1])
pass